# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [0]:
# try this project here!

In [0]:
!pip install syft

     |████████████████████████████████| 256kB 4.8MB/s 
     |████████████████████████████████| 460kB 42.9MB/s 
     |████████████████████████████████| 1.4MB 24.3MB/s 
     |████████████████████████████████| 389kB 32.9MB/s 
     |████████████████████████████████| 81kB 15.7MB/s 
     |████████████████████████████████| 204kB 39.5MB/s 
     |████████████████████████████████| 256kB 38.5MB/s 
     |████████████████████████████████| 266kB 40.5MB/s 
     |████████████████████████████████| 51kB 18.2MB/s 
     |████████████████████████████████| 122kB 45.4MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067089 sha256=e84ded54c3d77ea878561fa20e1c63cc3fde187ab25f273686e35e3cdeeedd61
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=9c8d35bae2aaab75baab40b72d3f44aae89e7da8e36566701d8ebb28a904cc4e
  Stored in di

In [51]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0811 12:37:41.148274 140215765444480 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure')

In [53]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0811 12:37:48.497667 140215765444480 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0811 12:37:48.502078 140215765444480 base.py:646] Worker secure already exists. Replacing old worker which could cause                     unexpected behavior
W0811 12:37:48.503909 140215765444480 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0811 12:37:48.507400 140215765444480 base.py:646] Worker secure already exists. Replacing old worker which could cause                     unexpected behavior
W0811 12:37:48.508947 140215765444480 base.py:646] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0811 12:37:48.510230 140215765444480 base.py:646] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure #objects:0>

In [0]:
# toy dataset
data =  th.tensor([[0., 1.], [1., 1.], [0., 0.], [1., 0]], requires_grad=True)
target =  th.tensor([[0.], [1.,], [0.,], [1.,]], requires_grad=True)

In [0]:
bob_data = data[0:2].send(bob)
bob_target = target[0:2].send(bob)

In [0]:
alice_data = data[2:].send(alice)
alice_target = target[2:].send(alice)

In [65]:
model = nn.Linear(2, 1)
model

Linear(in_features=2, out_features=1, bias=True)

In [0]:
bob_model = model.copy().send(bob)
alice_model = model.copy().send(alice)

In [0]:
bob_optim = optim.SGD(bob_model.parameters(), lr=0.01)
alice_optim = optim.SGD(alice_model.parameters(), lr=0.01)

In [59]:
for i in range(10):
  bob_optim.zero_grad()
  pred = bob_model(bob_data)
  loss = ((bob_target - pred) **2).sum()
  loss.backward()
  bob_optim.step()

  print(loss.get().data)

tensor(2.4518)
tensor(2.1603)
tensor(1.9171)
tensor(1.7140)
tensor(1.5440)
tensor(1.4014)
tensor(1.2814)
tensor(1.1801)
tensor(1.0943)
tensor(1.0214)


In [60]:
for i in range(10):
  alice_optim.zero_grad()
  pred = alice_model(alice_data)
  loss = ((alice_target - pred) **2).sum()
  loss.backward()
  alice_optim.step()

  print(loss.get().data)

tensor(1.9870)
tensor(1.8454)
tensor(1.7173)
tensor(1.6014)
tensor(1.4964)
tensor(1.4012)
tensor(1.3149)
tensor(1.2365)
tensor(1.1652)
tensor(1.1004)


In [0]:
bob_model.move(secure_worker)
alice_model.move(secure_worker)

In [62]:
secure_worker._objects

{11392493992: Parameter containing:
 tensor([0.3259], requires_grad=True), 12767298042: Parameter containing:
 tensor([[-0.2786,  0.0473]], requires_grad=True), 74767700206: Parameter containing:
 tensor([[-0.2844, -0.1606]], requires_grad=True), 76206637505: Parameter containing:
 tensor([0.3131], requires_grad=True)}

In [0]:
with th.no_grad():
    model.weight.set_(((alice_model.weight.data + bob_model.weight.data) / 2).get())
    model.bias.set_(((alice_model.bias.data + bob_model.bias.data) / 2).get())

In [73]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)

    bob_optim = optim.SGD(bob_model.parameters(), lr=0.01)
    alice_optim = optim.SGD(alice_model.parameters(), lr=0.01)
    
    for wi in range(worker_iters):
        # Train Bob's Model
        bob_optim.zero_grad()
        pred = bob_model(bob_data)
        b_loss = ((bob_target - pred) **2).sum()
        b_loss.backward()
        bob_optim.step()


        # Train Alice's Model
        alice_optim.zero_grad()
        pred = alice_model(alice_data)
        loss = ((alice_target - pred) **2).sum()
        loss.backward()
        alice_optim.step()

    bob_model.move(secure_worker)
    alice_model.move(secure_worker)

    with th.no_grad():
        model.weight.set_(((alice_model.weight.data + bob_model.weight.data) / 2).get())
        model.bias.set_(((alice_model.bias.data + bob_model.bias.data) / 2).get())
    
    print("Bob:" + str(b_loss.get().data) + " Alice:" + str(loss.get().data))

Bob:tensor(0.8396) Alice:tensor(0.8436)
Bob:tensor(0.6783) Alice:tensor(0.6846)
Bob:tensor(0.5753) Alice:tensor(0.5795)
Bob:tensor(0.5041) Alice:tensor(0.5056)
Bob:tensor(0.4506) Alice:tensor(0.4504)
Bob:tensor(0.4078) Alice:tensor(0.4066)
Bob:tensor(0.3715) Alice:tensor(0.3703)
Bob:tensor(0.3399) Alice:tensor(0.3392)
Bob:tensor(0.3116) Alice:tensor(0.3119)
Bob:tensor(0.2860) Alice:tensor(0.2874)


In [77]:
preds = model(data)
loss = ((preds - target) ** 2).sum()
loss

tensor(0.5648, grad_fn=<SumBackward0>)

In [78]:
print(preds)
print(target)
print(loss.data)

tensor([[0.4225],
        [0.6783],
        [0.4264],
        [0.6822]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [1.],
        [0.],
        [1.]], requires_grad=True)
tensor(0.5648)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [80]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [81]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [82]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [83]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [84]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
# try this project here!

In [0]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    shares = list()
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

def decrypt(shares):
    return sum(shares) % Q

In [87]:
shares = encrypt(3)
shares

(14184453092115380730780, 4898562787325554890096, 4657613964319303865850)

In [88]:
decrypt(shares)

3

In [0]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
x = encrypt(5)
y = encrypt(7)
z = add(x,y)

In [92]:
decrypt(z)

12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [100]:
encode(3.5)

35000

In [101]:
decode(35000)

3.5

In [102]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)

In [106]:
bob._objects

{24156271196: tensor([ 999837482941477172,  860410988676119102,  962208472797302351,
         1518512397399396633, 4386398065638592773])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [108]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:76037916251 -> bob:67681924852]
	-> [PointerTensor | me:14402033410 -> alice:82159028022]
	-> [PointerTensor | me:87961905946 -> secure:73252303803]
	*crypto provider: me*

In [109]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [111]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [113]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [115]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x

In [119]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation